In [8]:
from copy import deepcopy

import pandas as pd

from BitBoard import osero
from osero_learn import learn

PLAY_WAY = deepcopy(osero.PLAY_WAY)
del PLAY_WAY["human"]
PLAY_WAY = PLAY_WAY.values()

In [9]:
eva = [[
     1.0, -0.6,  0.6,  0.4,  0.4,  0.6, -0.6,  1.0,
    -0.6, -0.8,  0.0,  0.0,  0.0,  0.0, -0.8, -0.6,
     0.6,  0.0,  0.8,  0.6,  0.6,  0.8,  0.0,  0.6,
     0.4,  0.0,  0.6,  0.0,  0.0,  0.6,  0.0,  0.4,
     0.4,  0.0,  0.6,  0.0,  0.0,  0.6,  0.0,  0.4,
     0.6,  0.0,  0.8,  0.6,  0.6,  0.8,  0.0,  0.6,
    -0.6, -0.8,  0.0,  0.0,  0.0,  0.0, -0.8, -0.6,
     1.0, -0.6,  0.6,  0.4,  0.4,  0.6, -0.6,  1.0
] for i in range(2)]

In [15]:
df = pd.DataFrame({})

check_point = [i for i in range(1, 61)]

for i in range(10):
    print("\r[" + "#" * (i+1) + " " * (10-i+1) + "]", end="")
    for black in PLAY_WAY:
        for white in PLAY_WAY:
            run = learn(\
                black,
                white,
                check_point=check_point,
                seed_num=i,
                eva=eva
            )
            data = run.play()
            df = df.append(data, ignore_index=True)

print("\r[" + "#" * 10 + "]")

[##########]


In [16]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge

x_data = df.drop(["turn", "last_score"], axis=1)
y_data = df[["turn_num", "last_score"]]

turn_vari = df["turn_num"].unique()

In [17]:
model = []
alpha_arr = [i for i in range(3, 13)]

for turn_num in turn_vari:
    model.append([])

    for alpha in alpha_arr:
        x_train, x_test, y_train, y_test = train_test_split(\
            x_data.query("turn_num==%d" % turn_num).drop("turn_num", axis=1),
            y_data.query("turn_num==%d" % turn_num).drop("turn_num", axis=1),
            random_state=0
        )

        model[-1].append(Ridge(\
            alpha=alpha,
            solver="lbfgs",
            positive=True,
            random_state=0
        ))
        model[-1][-1].fit(x_train, y_train)

In [18]:
import pickle

for turn_num in range(len(turn_vari)):
    for alpha in range(len(alpha_arr)):
        with open(\
            "model/model%2d%2d.pickle" % (turn_vari[turn_num], alpha_arr[alpha]),
            mode="wb"
        ) as fp:
            pickle.dump(model[turn_num][alpha], fp)